<a href="https://colab.research.google.com/github/bachnguyenTE/temporal-mgn/blob/prototype-mgvae/temporal_mgvae_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Apr 26 00:04:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Arguments (inactive)

# def _parse_args():
#     parser = argparse.ArgumentParser(description = 'Temporal graph learning')
#     parser.add_argument('--dir', '-dir', type = str, default = '.', help = 'Directory')
#     parser.add_argument('--learning_target', '-learning_target', type = str, default = 'U0', help = 'Learning target')
#     parser.add_argument('--name', '-name', type = str, default = 'NAME', help = 'Name')
#     parser.add_argument('--dataset', '-dataset', type = str, default = 'ZINC_12k', help = 'ZINC')
#     parser.add_argument('--num_epoch', '-num_epoch', type = int, default = 2048, help = 'Number of epochs')
#     parser.add_argument('--batch_size', '-batch_size', type = int, default = 20, help = 'Batch size')
#     parser.add_argument('--learning_rate', '-learning_rate', type = float, default = 0.001, help = 'Initial learning rate')
#     parser.add_argument('--seed', '-s', type = int, default = 123456789, help = 'Random seed')
#     parser.add_argument('--n_clusters', '-n_clusters', type = int, default = 2, help = 'Number of clusters')
#     parser.add_argument('--n_levels', '-n_levels', type = int, default = 3, help = 'Number of levels of resolution')
#     parser.add_argument('--n_layers', '-n_layers', type = int, default = 3, help = 'Number of layers of message passing')
#     parser.add_argument('--hidden_dim', '-hidden_dim', type = int, default = 32, help = 'Hidden dimension')
#     parser.add_argument('--z_dim', '-z_dim', type = int, default = 32, help = 'Latent dimension')
#     parser.add_argument('--device', '-device', type = str, default = 'cpu', help = 'cuda/cpu')
#     args = parser.parse_args()
#     return args

# args = _parse_args()
# log_name = args.dir + "/" + args.name + ".log"
# model_name = args.dir + "/" + args.name + ".model"
# LOG = open(log_name, "w")

In [3]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
%%capture
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
!pip install torch-geometric-temporal

!pip install einops
!wget -c https://gist.githubusercontent.com/Luvata/55f7b3e9ae451122b9e3faf0a7387b4f/raw/440fac5c6e7153fd39e4eb9ebec6e51c9520ef1f/visualize.py
!pip install --upgrade graphviz

!pip install wandb -qqq
!pip install prettytable

In [4]:
# Library import (legacy MGVAE)
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, Adagrad
import pickle
from torch import optim
from torch.utils.data import DataLoader
import numpy as np
import os
import time
import argparse
from torch.nn import MultiheadAttention

# Library import (pytorch-geometric)
import torch_geometric 
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader, ClusterLoader, ClusterData
from torch_geometric.nn import MessagePassing, GCNConv, GATConv
from torch_geometric.utils import to_dense_adj, dense_to_sparse

###############################################################
# NOTE: 
# We preferably define our own clustering 
# procedure, rather than using the built-in ClusterLoader
# since there is a chance using ClusterLoader will not
# make the entire net differentiable (separate data process),
# and the net may no longer be isomorphic invariant.
###############################################################

from visualize import display_module
import wandb
import datetime
from tqdm import tqdm

In [5]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
wandb.init(project="temporal-mgn-attention-chickenpox", entity="temporal-graph-research")
wandb.config = {
    "learning_rate": 0.001,
    "epochs": 350,
    "batch_size": 1
}

wandb: Currently logged in as: temporal-graph-research (use `wandb login --relogin` to force relogin)


In [7]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: 
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [8]:
# Fix all random seed
torch_geometric.seed.seed_everything(69420)

# Set device to gpu
device = torch.device('cuda')

In [9]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.5)

In [10]:
# Define glorot initialization
def glorot_init(input_dim, output_dim):
    init_range = np.sqrt(6.0 / (input_dim + output_dim))
    initial = torch.rand(input_dim, output_dim) * 2 * init_range - init_range
    return nn.Parameter(initial)

In [11]:
# Multiresolution Graph Network
class MGN(nn.Module):
    def __init__(self, clusters, num_layers, node_dim, edge_dim, hidden_dim, z_dim, num_classes):
        super(MGN, self).__init__()
        self.clusters = clusters
        self.num_layers = num_layers
        self.node_dim = node_dim
        self.edge_dim = edge_dim
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim
        self.num_classes = num_classes

        self.base_encoder = GraphEncoder(self.num_layers, self.node_dim, self.edge_dim, self.hidden_dim, self.z_dim)

        self.cluster_learner = nn.ModuleList()
        self.global_encoder = nn.ModuleList()
        for i in range(len(self.clusters)):
            N = self.clusters[i]
            self.cluster_learner.append(GraphCluster(self.num_layers, self.z_dim, self.hidden_dim, N))
            if edge_dim is not None:
                self.global_encoder.append(GraphEncoder(self.num_layers, self.z_dim, self.hidden_dim, self.hidden_dim, self.z_dim))
            else:
                self.global_encoder.append(GraphEncoder(self.num_layers, self.z_dim, None, self.hidden_dim, self.z_dim))

        self.vertical_attention = MultiheadAttention(self.z_dim, 1, batch_first=True)

        D = self.z_dim * (len(self.clusters) + 1)
        self.fc1 = nn.Linear(self.z_dim, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, adj, node_feat, edge_feat = None):
        outputs = []

        # Base encoder
        base_latent, base_edge_hidden = self.base_encoder(adj, node_feat, edge_feat)

        outputs.append([base_latent, adj, base_edge_hidden])

        l = 0
        while l < len(self.clusters):
            if l == 0:
                prev_adj = adj
                prev_latent = base_latent
                prev_edge_hidden = base_edge_hidden
            else:
                prev_adj = outputs[len(outputs) - 1][1]
                prev_latent = outputs[len(outputs) - 1][0]
                prev_edge_hidden = outputs[len(outputs) - 1][2]

            # Assignment score
            assign_score = self.cluster_learner[l](prev_adj, prev_latent)

            # Softmax (soft assignment)
            # assign_matrix = F.softmax(assign_score, dim = 2)

            # Gumbel softmax (hard assignment)
            assign_matrix = F.gumbel_softmax(assign_score, tau = 1, hard = True, dim = 1)

            # Print out the cluster assignment matrix
            # print(torch.sum(assign_matrix, dim = 0))

            # Shrinked latent
            shrinked_latent = torch.matmul(assign_matrix.transpose(0, 1), prev_latent)

            # Latent normalization
            shrinked_latent = F.normalize(shrinked_latent, dim = 0)

            # Shrinked adjacency
            # print(f'Iteration: {l}')
            # print(to_dense_adj(prev_adj))
            # print(f'to_dense_adj size: {to_dense_adj(prev_adj, max_num_nodes=self.clusters[l]).size()}')
            # print(f'assign_matrix size: {assign_matrix.size()}')
            # print(f'node_feat size: {node_feat.size()}')
            # print(f'adj size: {prev_adj.size()}')
            # print(f'prev_adj size: {prev_adj.size()}')
            if l == 0:
                shrinked_adj = torch.einsum('sdf,st->tdf', torch.matmul(assign_matrix.transpose(0, 1), to_dense_adj(prev_adj, edge_attr = prev_edge_hidden, max_num_nodes=node_feat.size()[0])[0]), assign_matrix)
            else:
                shrinked_adj = torch.einsum('sdf,st->tdf', torch.matmul(assign_matrix.transpose(0, 1), to_dense_adj(prev_adj, edge_attr = prev_edge_hidden, max_num_nodes=self.clusters[l - 1])[0]), assign_matrix)
                # shrinked_adj = torch.matmul(torch.matmul(assign_matrix.transpose(0, 1), to_dense_adj(prev_adj, edge_attr=prev_edge_hidden, max_num_nodes=self.clusters[l - 1])[0]), assign_matrix)

            # Adjacency normalization
            shrinked_adj = shrinked_adj / torch.sum(shrinked_adj)

            # Reformatting adjacency matrix as edge index
            shrinked_edge_indices = torch.stack(torch.sum(shrinked_adj, dim=2).nonzero(as_tuple=True), dim=0)
            shrinked_edge_hidden = shrinked_adj[shrinked_edge_indices[0], shrinked_edge_indices[1], :]

            # Global encoder
            next_latent, next_edge_hidden = self.global_encoder[l](shrinked_edge_indices, shrinked_latent, shrinked_edge_hidden)

            outputs.append([next_latent, shrinked_edge_indices, next_edge_hidden])
            l += 1

        # Scalar prediction with vertical attention
        latents = torch.stack([torch.sum(output[0], dim = 0) for output in outputs], dim = 0)
        latents = latents[None, :]
        att_latents, _ = self.vertical_attention(latents, latents, latents)
        output_latents = torch.mean(att_latents[0], dim=0)
        hidden = torch.tanh(self.fc1(output_latents))
        predict = self.fc2(hidden)

        return predict, output_latents, outputs

In [12]:
# Graph encoder block
class GraphEncoder(nn.Module):
    def __init__(self, num_layers, node_dim, edge_dim, hidden_dim, z_dim, use_concat_layer=True, **kwargs):
        super(GraphEncoder, self).__init__(**kwargs)
        self.num_layers = num_layers
        self.node_dim = node_dim
        self.edge_dim = edge_dim
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim
        self.use_concat_layer = use_concat_layer

        self.node_fc1 = nn.Linear(self.node_dim, 128)
        self.node_fc2 = nn.Linear(128, self.hidden_dim)

        if self.edge_dim is not None:
            self.edge_fc1 = nn.Linear(self.edge_dim, 128)
            self.edge_fc2 = nn.Linear(128, self.hidden_dim)

        self.base_net = nn.ModuleList()
        self.combine_net = nn.ModuleList()
        for layer in range(self.num_layers):
            self.base_net.append(GATConv(self.hidden_dim, self.hidden_dim, edge_dim=self.hidden_dim))

        if self.use_concat_layer == True:
            self.latent_fc1 = nn.Linear((self.num_layers + 1) * self.hidden_dim, 256)
            self.latent_fc2 = nn.Linear(256, self.z_dim)
        else:
            self.latent_fc1 = nn.Linear(self.hidden_dim, 256)
            self.latent_fc2 = nn.Linear(256, self.z_dim)

    def forward(self, adj, node_feat, edge_feat=None):
        node_hidden = torch.tanh(self.node_fc1(node_feat))
        node_hidden = torch.tanh(self.node_fc2(node_hidden))

        if edge_feat is not None and self.edge_dim is not None:
            edge_hidden = torch.tanh(self.edge_fc1(edge_feat))
            edge_hidden = torch.tanh(self.edge_fc2(edge_hidden))
        else:
            edge_hidden = None

        all_hidden = [node_hidden]

        if edge_feat is not None:
            for layer in range(len(self.base_net)):
                if layer == 0:
                    hidden = self.base_net[layer](node_hidden, adj, edge_hidden)
                else:
                    hidden = self.base_net[layer](hidden, adj, edge_hidden)
            
                all_hidden.append(hidden)
        else:
            for layer in range(len(self.base_net)):
                if layer == 0:
                    hidden = self.base_net[layer](node_hidden, adj)
                else:
                    hidden = self.base_net[layer](hidden, adj)
            
                all_hidden.append(hidden)

        if self.use_concat_layer == True:
            hidden = torch.cat(all_hidden, dim=1)

        latent = torch.tanh(self.latent_fc1(hidden))
        latent = torch.tanh(self.latent_fc2(latent))
        return latent, edge_hidden

In [13]:
# Graph clustering block
class GraphCluster(nn.Module):
    def __init__(self, num_layers, input_dim, hidden_dim, z_dim, **kwargs):
        super(GraphCluster, self).__init__(**kwargs)
        self.num_layers = num_layers
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim

        self.fc1 = nn.Linear(self.input_dim, 128)
        self.fc2 = nn.Linear(128, self.hidden_dim)

        # Option 1: Learnable clustering
        self.base_net = nn.ModuleList()
        
        # Option 2: Fixed clustering
        # self.base_net = []

        for layer in range(self.num_layers):
            self.base_net.append(GCNConv(self.hidden_dim, self.hidden_dim))

        self.assign_net = GCNConv(self.hidden_dim, self.z_dim)

    def forward(self, adj, X):
        hidden = torch.sigmoid(self.fc1(X))
        hidden = torch.sigmoid(self.fc2(hidden))
        for net in self.base_net:
            hidden = net(hidden, adj)
        assign = self.assign_net(hidden, adj)
        return assign

In [14]:
class TemporalMGN(nn.Module):
    def __init__(self, clusters, num_layers, node_dim, edge_dim, hidden_dim, z_dim, num_classes, time_seq_length):
        super(TemporalMGN, self).__init__()
        self.clusters = clusters
        self.num_layers = num_layers
        self.node_dim = node_dim
        self.edge_dim = edge_dim
        self.hidden_dim = hidden_dim
        self.z_dim = z_dim
        self.num_classes = num_classes
        self.time_seq_length = time_seq_length

        self.attention_multires = MGN(self.clusters, self.num_layers, self.node_dim, self.edge_dim, self.hidden_dim, self.z_dim, self.num_classes)
        self.horizontal_attention = MultiheadAttention(self.z_dim, 1, batch_first=True)

        self.fc1 = nn.Linear(self.z_dim, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, time_step: int, graph_seq_embed, graph_seq_mask, adj, node_feat, edge_feat=None):
        _, multires_latents, _ = self.attention_multires(adj, node_feat, edge_feat)
        if time_step is not 0:
            graph_seq_embed = torch.cat([graph_seq_embed, multires_latents[None, :]], dim=0)
        else:
            graph_seq_embed = multires_latents[None, :]

        attn_embed = graph_seq_embed[None, :].to(device)
        horizontal_attn_latents, _ = self.horizontal_attention(attn_embed, attn_embed, attn_embed)
        output_latents = torch.mean(horizontal_attn_latents[0], dim=0)
        hidden = torch.tanh(self.fc1(output_latents))
        predict = self.fc2(hidden)

        return predict, multires_latents[None, :]

In [15]:
# adj is the adjacency matrix
# PyG: need to convert graph data format to the adjacency matrix format, or rewrite code

In [16]:
time_steps_count = 258
mask_tensor = torch.empty((time_steps_count, time_steps_count, time_steps_count))
print(mask_tensor.size())

for i in range(time_steps_count):
    time_len = i + 1
    mask = (torch.triu(torch.ones(time_len, time_len)) == 1).transpose(0, 1).float()
    mask = F.pad(mask, (0, time_steps_count-time_len, 0, time_steps_count-time_len), "constant", 0)
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    mask_tensor[i] = mask

torch.Size([258, 258, 258])


In [17]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = TemporalMGN(
    clusters=[4],
    num_layers=4,
    node_dim=4,
    edge_dim=1,
    hidden_dim=64,
    z_dim=16,
    num_classes=20,
    time_seq_length = time_steps_count
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
param_count = count_parameters(model)
model.train()

for epoch in range(350):
    cost = 0
    graph_seq_embed_list = []
    masking = mask_tensor
    for time, snapshot in enumerate(train_dataset):
        masking.to(device)
        snapshot.to(device)
        if time == 0:
            graph_seq_embed = torch.empty((0, 0)).to(device)
        else: 
            graph_seq_embed = torch.cat(graph_seq_embed_list, dim=0).to(device)
        y_hat, cur_graph_seq_embed = model(time, graph_seq_embed, masking[time].to(device), snapshot.edge_index, snapshot.x, snapshot.edge_attr[:, None])
        # print(f"y_hat size: {y_hat.size()}")
        # print(f"y_hat: {y_hat}")
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
        graph_seq_embed_list.append(cur_graph_seq_embed)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    wandb.log({"train_loss": cost})
    print(f'Epoch: {epoch:03d}, Train Loss: {cost:.4f}')

<IPython.core.display.Javascript object>

+----------------------------------------------------------------+------------+
|                            Modules                             | Parameters |
+----------------------------------------------------------------+------------+
|        attention_multires.base_encoder.node_fc1.weight         |    512     |
|         attention_multires.base_encoder.node_fc1.bias          |    128     |
|        attention_multires.base_encoder.node_fc2.weight         |    8192    |
|         attention_multires.base_encoder.node_fc2.bias          |     64     |
|        attention_multires.base_encoder.edge_fc1.weight         |    128     |
|         attention_multires.base_encoder.edge_fc1.bias          |    128     |
|        attention_multires.base_encoder.edge_fc2.weight         |    8192    |
|         attention_multires.base_encoder.edge_fc2.bias          |     64     |
|       attention_multires.base_encoder.base_net.0.att_src       |     64     |
|       attention_multires.base_encoder.

In [18]:
model.eval()
cost = 0
masking = mask_tensor
for time, snapshot in enumerate(test_dataset):
    if time == 257:
        break
    masking.to(device)
    snapshot.to(device)
    if time == 0:
        graph_seq_embed = torch.empty((0, 0)).to(device)
    else: 
        graph_seq_embed = torch.cat(graph_seq_embed_list, dim=0).to(device)
    y_hat, cur_graph_seq_embed = model(time, graph_seq_embed, masking[time].to(device), snapshot.edge_index, snapshot.x, snapshot.edge_attr[:, None])
    # print(f"y_hat size: {y_hat.size()}")
    # print(f"y_hat: {y_hat}")
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
    graph_seq_embed_list.append(cur_graph_seq_embed)
cost = cost / (time)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 0.9073


In [19]:
from google.colab import files

wandb.save("mgn_attention_qm9_1_res_4_clusters.h5")
torch.save(model.state_dict(), "mgn_attention_qm9_1_res_4_clusters.pth")
files.download("mgn_attention_qm9_1_res_4_clusters.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Deprecated code

In [20]:
# # Dataset testing
# from torch_geometric.datasets import ZINC, QM9, TUDataset, KarateClub, GNNBenchmarkDataset

# # dataset = ZINC('data/ZINC')
# dataset = QM9(root='data/QM7b')
# # dataset = TUDataset(root='data/TUDataset', name='PROTEINS')
# # dataset = KarateClub()
# # dataset = GNNBenchmarkDataset(root='data/GNNBenchmarkDataset', name='MNIST')

# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# data = dataset[0] # Get the first graph object

# print()
# print(data)
# print('=============================================================')

# # Gather some statistics about the first graph
# print(f'Number of nodes: {data.num_nodes}')
# print(f'Number of edges: {data.num_edges}')
# print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# print(f'Has isolated nodes: {data.has_isolated_nodes()}')
# print(f'Has self-loops: {data.has_self_loops()}')
# print(f'Is undirected: {data.is_undirected()}')

In [21]:
# dataset = dataset.shuffle()

# train_dataset = dataset[:10000]
# test_dataset = dataset[10000:11000]

# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [22]:
# # Minibatching the dataset 
# from torch_geometric.loader import DataLoader

# train_loader = DataLoader(train_dataset, shuffle=True)
# test_loader = DataLoader(test_dataset, shuffle=False)

# # Compute the data mean and standard deviation
# all_y = []
# for data in train_loader:
#   all_y.append(data.y.detach().numpy())
# for data in test_loader:
#   all_y.append(data.y.detach().numpy())
# all_y = np.concatenate(all_y)
# mean_y = np.mean(all_y)
# std_y = np.std(all_y)
# print('Mean:', mean_y)
# print('STD:', std_y)

In [23]:
# from IPython.display import Javascript
# display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

# model = MGN(
#     clusters=[4],
#     num_layers=4,
#     node_dim=dataset.num_features,
#     edge_dim=dataset.num_edge_features,
#     hidden_dim=64,
#     z_dim=16,
#     num_classes=dataset.num_classes
# ).to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
# criterion = torch.nn.MSELoss(reduction='mean')

# param_count = count_parameters(model)

# def train():
#     model.train()

#     for data in train_loader:  # Iterate in batches over the training dataset.
#         data.to(device)
#         predict, latent, outputs = model(data.edge_index, data.x, data.edge_attr)  # Perform a single forward pass.

#         # Normalization
#         new_y = (data.y.detach().cpu() - mean_y) / std_y
#         new_y = new_y.to(device)

#         # loss = criterion(predict[None, :], data.y)  # Compute the loss.
#         loss = criterion(predict[None, :], new_y)  # Compute the loss.

#         loss.backward()  # Derive gradients.
#         optimizer.step()  # Update parameters based on gradients.
#         optimizer.zero_grad()  # Clear gradients.

# def test(loader):
#     model.eval()

#     correct = 0
#     for data in loader:  # Iterate in batches over the training/test dataset.
#         data.to(device)
#         predict, latent, outputs = model(data.edge_index, data.x, data.edge_attr)  

#         # Normalization
#         new_y = (data.y.detach().cpu() - mean_y) / std_y
#         new_y = new_y.to(device)

#         # modelLoss = criterion(predict[None, :], data.y)
#         modelLoss = criterion(predict[None, :], new_y)

#     return modelLoss  # Derive ratio of correct predictions.


# for epoch in range(200):
#     train()
#     train_loss = test(train_loader)
#     test_loss = test(test_loader)
#     wandb.log({"train_loss": train_loss, "test_loss": test_loss})
#     print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')